# Start the MONAILabel server for local access
Now that we have customized the MONAILabel segmentation config, we can locally deploy the MONAILabel server and try it out. In a terminal window, navigate to the directory containing this notebook and run:

```bash
monailabel start_server --app apps/radiology --studies nifti_resampled --conf models segmentation_globes
```
### Explanation of options
* `--app apps/radiology` - this is the location of your radiology app (downloaded earlier in this document). This can be moved to any location you desire.
* `--studies nifti_resampled` - this is the location of your data. In this case, we are using the `nifti_resampled` directory that we created in the `dicom2deployment-data-prep.ipynb` document.
* `--conf models segmentation_globes` - this is the location of our custom segmentation config (created earlier in this notebook).

If we want to enable epistemic loss calculation (for active learning) then we need to add a few more options to the start_server call. This is only relevant once you have already trained at least one initial model:

```bash
monailabel start_server --app apps/radiology --studies nifti_resampled --conf models segmentation_globes --conf skip_scoring false --conf skip_strategies false --conf epistemic_enabled true
```
### Explanation of additional config options for epistemic uncertainty
* `--conf skip_scoring false` - this tells MONAILabel to score each of the unlabeled studies using the `scoring_method` we defined previously.
* `--conf skip_strategies false` - this enables the active learning `strategy` method that we defined previously.
* `--conf epistemic_enabled true` - this enables epistemic scoring.

# Deployment using HTTPS
You may wish to deploy your server using HTTPS to enhance security. To accomplish this, you will have to self-sign a certificate using openssl and launch the server in HTTPS mode.
```bash
# Create self-signed ssl cert
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout uvicorn-selfsigned.key -out uvicorn-selfsigned.crt

# Start server in ssl mode
monailabel start_server --app apps/radiology --studies nifti_resampled --conf models segmentation_globes --conf skip_scoring false --conf skip_strategies false --conf epistemic_enabled true --ssl_keyfile uvicorn-selfsigned.key --ssl_certfile uvicorn-selfsigned.crt
```

# Using 3D Slicer to access the local MONAILabel server
Now that we are running the MONAILabel server on our local machine, we can access it locally using 3D Slicer. Detailed instructions for installing 3D Slicer and accessing the local server can be found here:

https://docs.monai.io/projects/label/en/latest/quickstart.html#install-monai-label-plugin-in-3d-slicer

If you deployed your server with HTTPS enabled, then you must include https:// at the beginning of your server address. If you did not use HTTPS mode, then use http:// instead.

# Accessing MONAILabel server on your local area network (LAN)
The following (optional) steps will allow other people on your local area network to access the MONAILabel server and collaborate on the active learning segmentation task.

## <span style="color:red">CAUTION: SECURITY RISK</span>
<span style="color:red">Please do not attempt the follow steps without consulting an IT/networking professional. The following steps, particularly opening a port, can expose your computer to unauthorized access.</span>

## Opening a port
By default, MONAILabel runs on port 8000. This can be changed using the `--port` flag to the `start_server` command above. To open this port, you can use one of the following methods:
* Ubuntu-based Linux: `sudo ufw allow 4000`
* Cent-OS-based Linux: `firewall-cmd --add-port=4000/tcp`
* Windows: https://learn.microsoft.com/en-us/sql/reporting-services/report-server/configure-a-firewall-for-report-server-access?view=sql-server-ver16 (use port 8000 instead of port 80 as written)
* MacOS: https://www.macworld.com/article/671729/mac-firewall-how-to-open-specific-ports-in-os-x-10-10-firewall.html (use port 8000)

## Determine your private IP
In order for others in your local area network to access your MONAILabel server, you will need to provide your private IP address. This can be found on most systems using the `ipconfig` or `ifconfig` commands. You are looking for something like this: `inet 192.168.86.25`. 

## Connecting to the MONAILabel server over the local network
Using the private IP address from above example, users would be able to connect to your MONAILabel server using the following address:

`http://192.168.86.25:8000`

![Server IP Entry](img/slicer.jpeg)

# Accessing your MONAILabel server through the internet
The following (optional) steps will allow anyone in the world to access the MONAILabel server through the internet and collaborate on the active learning segmentation task.

## <span style="color:red">CAUTION: SECURITY RISK</span>
<span style="color:red">Please do not attempt the follow steps without consulting an IT/networking professional. The following steps, if configured incorrectly, can expose your computer to unauthorized access.</span>

## Creating a secure tunnel to your server using Cloudflare
It is possible to create a secure, "zero-trust" tunnel to your locally deployed MONAILabel server so that anyone on the internet can access it through a domain that you own. To do this you will need the following:
* an internet domain that you own and administrate
* a free account on Cloudflare
* complete the steps above for enabling HTTPS mode

This YouTube video provides a broad overview of setting up a cloudflare tunnel:
https://www.youtube.com/watch?v=ZvIdFs3M5ic
This video describes setting up a secure tunnel for a NAS, but just replace NAS with your MONAILabel server.

Briefly, you will do the following:
* add your domain to Cloudflare
* create a Cloudflare tunnel in the "zero trust dashboard"
* install the Cloudflare connector software on the computer that is hosting your MONAILabel server using Cloudflared or Docker
* create a route for your tunnel
    * make a subdomain (e.g. monai.your-domain.com)
    * connect this to the relevant local service (HTTP or HTTPS) using your complete IP address and port
    * save the tunnel route to activate it
* once this is complete, you should be able to access your MONAILabel server from anywhere using your subdomain URL (e.g. monai.<your-domain>.com)

<span style="color:red">Please note that anyone in the world with your URL can access your server. There are many options for limiting access to your tunnel within the Cloudflare zero trust dashboard including limiting access to specific IP addresses. These measures are highly recommended for security reasons.</span>

# Deploying a MONAILabel server on AWS
Another easier and more secure method for deploying a MONAILabel server is the AWS cloud. AWS has pre-built solutions for scalable MONAILabel deployment. For more information please see:

https://aws.amazon.com/blogs/industries/ai-assisted-annotation-of-medical-images-using-monai-label-on-aws/

If you are interested, you can get in touch with an AWS sales representative here:

https://pages.awscloud.com/HealthContactSales.html?languages=english